In [ ]:
import re
import math
import pandas as pd
import string
import nltk
import spacy
import numpy as np
from collections import Counter, defaultdict
from nltk.tokenize import TweetTokenizer

In [ ]:
def tokenize(data):
    text = []
    for i in range(len(data)):
        tmp = tknzr.tokenize(data['text'][i].lower())
        tmp = ['<s>'] + tmp + ['</s>']
        text.append(tmp)
    return text 

In [ ]:
def get_bigrams(text_list):
    new_text_list = []
    for text in text_list:
        new_text_list.append(list(nltk.bigrams(text)))
    return new_text_list

In [ ]:
def get_terms(text_list):
    vocabulary = Counter()
    for text in text_list:
        vocabulary.update(text)
    vocabulary = Counter(el for el in vocabulary.elements() if vocabulary[el] >= 3)
    return vocabulary

In [ ]:
def replace_UNK(text_list):
    valid_tokens = 0
    new_text_list = []
    for text in text_list:
        new_text = []
        for i in range(len(text)):
            if text[i] in list(vocabulary):
                new_text.append(text[i])
            else:
                new_text.append('<UNK>')
        new_text_list.append(new_text)
    return new_text_list

In [ ]:
def get_dict():
    for bigrams in training_text_bigrams:
        for w1, w2 in bigrams:
            counts[w1][w2] += 1

In [ ]:
def get_reverse_dict():
    for bigrams in training_text_bigrams:
        for w2, w1 in bigrams:
            reverse_counts[w1][w2] += 1

In [ ]:
def get_average_perplexity():
    training_perplexity = 0
    for sentence in training_text_bigrams:
        training_probabilities = [(1 + counts[w1][w2])/(sum(counts[w1].values()) + len(vocabulary)) for w1, w2 in sentence]
        N = len(sentence)
        cross_entropy = -1/N * sum([math.log(p, 2) for p in training_probabilities])
        training_perplexity += math.pow(2, cross_entropy)
    print("average training perplexity:",training_perplexity / len(training_text_bigrams))
    testing_perplexity = 0
    for sentence in testing_text_bigrams:
        testing_probabilities = [(1 + counts[w1][w2])/(sum(counts[w1].values()) + len(vocabulary)) for w1, w2 in sentence]
        N = len(sentence)
        cross_entropy = -1/N * sum([math.log(p, 2) for p in testing_probabilities])
        testing_perplexity += math.pow(2, cross_entropy)
    print("average testing perplexity: ",testing_perplexity / len(testing_text_bigrams))

In [ ]:
def get_best_average_perplexity():
    best_testing_average_perplexity = 1e9
    best_gamma = 0
    for gamma in gamma_list:
        testing_perplexity = 0
        for sentence in testing_text_bigrams:
            testing_probabilities = [(gamma * (1 + counts[fwd[0]][fwd[1]])/(sum(counts[fwd[0]].values()) + len(vocabulary)) 
                                      + (1-gamma) * (1 + reverse_counts[bkd[1]][bkd[0]])/(sum(reverse_counts[bkd[1]].values()) + len(vocabulary))) for fwd, bkd in zip(sentence[:-1], sentence[1:])]
            N = len(sentence)
            cross_entropy = -1/N * sum([math.log(p, 2) for p in testing_probabilities])
            testing_perplexity += math.pow(2, cross_entropy)
        testing_average_perplexity = testing_perplexity / len(testing_text_bigrams)
        if testing_average_perplexity < best_testing_average_perplexity:
            best_testing_average_perplexity = testing_average_perplexity
            best_gamma = gamma
    training_perplexity = 0
    for sentence in training_text_bigrams:
        training_probabilities = [(best_gamma * (1 + counts[fwd[0]][fwd[1]])/(sum(counts[fwd[0]].values()) + len(vocabulary)) + (1-best_gamma) * (1 + reverse_counts[bkd[1]][bkd[0]])/(sum(reverse_counts[bkd[1]].values()) + len(vocabulary))) for fwd, bkd in zip(sentence[:-1], sentence[1:])]
        N = len(sentence)
        cross_entropy = -1/N * sum([math.log(p, 2) for p in training_probabilities])
        training_perplexity += math.pow(2, cross_entropy)
    print("best gamma: ",best_gamma)
    print("average training perplexity: ", training_perplexity / len(training_text_bigrams))
    print("average testing perplexity: ", best_testing_average_perplexity)

In [ ]:
testing_data = pd.read_json("https://raw.githubusercontent.com/bshmueli/108-nlp/master/tweets_test.txt", lines=True)
training_data = pd.read_json("https://raw.githubusercontent.com/bshmueli/108-nlp/master/tweets_train.txt", lines=True)
tknzr = TweetTokenizer()

training_token = tokenize(training_data)
testing_token = tokenize(testing_data)

vocabulary = get_terms(training_token)

training_text = replace_UNK(training_token)
testing_text = replace_UNK(testing_token)

training_text_bigrams = get_bigrams(training_text)
testing_text_bigrams = get_bigrams(testing_text)

In [ ]:
counts = defaultdict(lambda: defaultdict(lambda: 0))

get_dict()
print("PART 1")
get_average_perplexity()

PART 1
average training perplexity: 1355.335648828969
average testing perplexity:  1640.5500188061494


In [ ]:
reverse_counts = defaultdict(lambda: defaultdict(lambda: 0))
get_reverse_dict()

In [ ]:
gamma_list = [round(x * 0.05, 2) for x in range(0, 21)]
print("PART 2")
get_best_average_perplexity()

PART 2
best gamma:  0.5
average training perplexity:  654.3808701553082
average testing perplexity:  744.4107434171606
